### Setup your environnement variables

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["UIFORM_API_KEY"] = "YOUR API KEY" # go to https://uiform.com to create your API Key

# Get started

In [5]:
from dotenv import load_dotenv
assert load_dotenv(override=True) # Load environment variables from a .env file

In [ ]:
from uiform import AsyncUiForm
uiclient = AsyncUiForm()
await uiclient.models.list()

# Example 1 - Analyze informations from an email

#### Option A - You use our preprocess endpoint and openAI client

In [ ]:
import json
from uiform import AsyncUiForm, Schema
from openai import AsyncOpenAI

with open("freight/booking_confirmation_json_schema.json", "r") as f:
    json_schema = json.load(f)


uiclient = AsyncUiForm()
doc_msg = await uiclient.documents.create_messages(
    document = "freight/booking_confirmation.jpg",
)

schema_obj = Schema(
    json_schema = json_schema
)


# Now you can use your favorite model to analyze your document
client = AsyncOpenAI()
completion = await client.beta.chat.completions.parse(
    model="gpt-4o-mini", 
    messages=schema_obj.openai_messages + doc_msg.openai_messages,
    response_format=schema_obj.inference_pydantic_model
)

# Validate the response against the original schema if you want to remove the reasoning fields
assert completion.choices[0].message.content is not None
extraction = schema_obj.pydantic_model.model_validate_json(
    completion.choices[0].message.content
)

print(extraction)

#### Option B - Using UiForm `extraction.parse` endpoint

In [ ]:
import json
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()
completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
)

print(completion.choices[0].message.parsed)

#### Option C - Using UiForm `extraction.stream` endpoint

In [ ]:
import json
from uiform.client import UiForm
from IPython.display import clear_output, display

uiclient = AsyncUiForm()

async for completion in uiclient.documents.extractions.stream(
    json_schema = "freight/schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini",
    temperature=0
):
    clear_output(wait=True)
    if completion.choices[0].message.parsed is not None:
        display(completion.choices[0].message.parsed.model_dump())

## Appendix A - Use text as modality

In [ ]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document="freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    modality='text'
)

print(completion.choices[0].message.parsed)

## Appendix B - Add additional context

In [ ]:
from uiform.client import AsyncUiForm

uiclient = AsyncUiForm()

text_operations = {'regex_instructions' : [
        {
            "name": "vat_number",
            "pattern": r"[Ff][Rr]\s*(\d\s*){11}",
            "description": "VAT number in the format XX999999999"
        }
    ]
}

completion = await uiclient.documents.extractions.parse(
    json_schema = "freight/booking_confirmation_json_schema.json",
    document = "freight/booking_confirmation.jpg",
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    text_operations=text_operations
)

print(completion.choices[0].message.parsed)